In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd
import time

In [2]:
gdf = gpd.read_file("../../../TX/geographic/County/TX_County_Cleaned.json")
gdf.head(1)

,County,geometry
0,Uvalde County,"POLYGON ((-100.11112 29.08625, -100.11122 29.1..."


In [7]:
len(gdf["County"].unique())

254

In [9]:
counties = list(gdf["County"].unique())
len(counties)

254

In [12]:
file_name = "TX7County.json"
g = gdf[gdf["County"].isin(counties[7*31:])]
g.to_file(file_name, driver="GeoJSON")

In [11]:
len(gdf[gdf["County"].isin(counties[7*31:])])

37

In [10]:
254//8

31

In [13]:
!ls

TX0Precinct.json     TX4Precinct.json     TX7Precinct.json
TX1Precinct.json     TX5Precinct.json     parse_precinct.ipynb
TX2Precinct.json     TX6Precinct.json
TX3Precinct.json     TX7County.json


# create x y for each polygon

In [29]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [30]:
# create a new df
# columns = ["index", "shape_index", "polygon_index", "Precinct #", "County", "Precinct Name", "Hillary Clinton/Dem",
#            "Donald J. Trump/Rep", "Gary Johnson/Lib", "Jill Stein/Grn", "Other/Other",
#           "Total Votes", "X_Coord", "Y_Coord"]

In [36]:
def parse(gdf, geoID, shapeID, polyID):
    columns = ["geovertex_id", "shape_id", "polygon_id", "CName", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = geoID
    shape_id = shapeID
    polygon_id = polyID

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%25 == 0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [37]:
geovertex_id = 419894
polygon_id = 940
shape_id = 1

for i in range(8):
    FILE_NAME = "TX" + str(i) + "Precinct.json"
    gdf = gpd.read_file(FILE_NAME)
    
    # drop some columns
    dropping = gdf.columns.to_list()
    dropping.remove("geometry")
    dropping.remove("CName")
    gdf.drop(dropping, inplace=True, axis=1)
    
    # start parsing
    print("\nStart Parsing: {}".format(FILE_NAME))
    start_time = time.time()
    gdf_Parsed = parse(gdf, geovertex_id, shape_id, polygon_id)
    end_time = time.time()
    print("\nTotal Time: {} mins".format((end_time-start_time)//60))
    
    # save the parsed file
    name = FILE_NAME.split(".")[0]
    gdf_Parsed.to_csv(name+"_Parsed.csv", index=False)
    
    # update id
    shape_id = gdf_Parsed.iloc[-1]["shape_id"]+1
    polygon_id = gdf_Parsed.iloc[-1]["polygon_id"]+1
    geovertex_id = gdf_Parsed.iloc[-1]["geovertex_id"]+1


Start Parsing: TX0Precinct.json
 0.00% ###
Total Time: 0.0 mins

Start Parsing: TX1Precinct.json
 0.00% ###
Total Time: 0.0 mins

Start Parsing: TX2Precinct.json
 0.00% ###
Total Time: 0.0 mins
